In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import tensorflow as tf
import tensorflow.keras as kr

In [2]:
batch_size = 32
img_height = 180
img_width = 180

In [3]:
train_ds = kr.utils.image_dataset_from_directory(
    "COVID-19_Radiography_Dataset",
    subset = "training",
    validation_split = 0.2,
    seed = 123,
    image_size = (img_height,img_width)
)
test_ds = kr.utils.image_dataset_from_directory(
    "COVID-19_Radiography_Dataset",
    subset = "validation",
    validation_split = 0.2,
    seed = 123,
    image_size = (img_height,img_width)
)

Found 15153 files belonging to 3 classes.
Using 12123 files for training.
Found 15153 files belonging to 3 classes.
Using 3030 files for validation.


In [4]:
train_ds = kr.utils.image_dataset_from_directory(
    "Covid19-dataset/train",
    seed = 123,
    image_size = (img_height,img_width)
)
test_ds = kr.utils.image_dataset_from_directory(
    "Covid19-dataset/test/",
    seed = 123,
    image_size = (img_height,img_width)
)

Found 251 files belonging to 3 classes.
Found 66 files belonging to 3 classes.


In [5]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().prefetch(buffer_size = AUTOTUNE)
val_ds = test_ds.cache().prefetch(buffer_size = AUTOTUNE)

In [38]:
base_model = tf.keras.applications.Xception(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=(180,180,3),
    pooling="avg",
    classes=3,
    classifier_activation="softmax"
)



In [39]:
base_model.trainable = False

In [40]:
inputs = kr.Input(shape=(180,180,3))
mean = np.array([127.5]*3)
var = mean ** 2

norm_layer = kr.layers.experimental.preprocessing.Normalization(mean=mean,variance=var)
x = norm_layer(inputs)

x = base_model(x)

x = kr.layers.Dense(256)(x)
x = kr.layers.Dense(64)(x)
output = kr.layers.Dense(3, activation="softmax")(x)
model_fin = kr.Model(inputs,output)

In [41]:
model_fin.summary()


Model: "model_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_10 (InputLayer)       [(None, 180, 180, 3)]     0         
                                                                 
 normalization_5 (Normalizat  (None, 180, 180, 3)      0         
 ion)                                                            
                                                                 
 xception (Functional)       (None, 2048)              20861480  
                                                                 
 dense_15 (Dense)            (None, 256)               524544    
                                                                 
 dense_16 (Dense)            (None, 64)                16448     
                                                                 
 dense_17 (Dense)            (None, 3)                 195       
                                                           

In [42]:
optimzer = kr.optimizers.Adam(learning_rate=0.000001)
model_fin.compile(
    optimizer="Adam",
    loss = tf.keras.losses.SparseCategoricalCrossentropy(),
    metrics=["accuracy"]
)

In [43]:
from keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=2)


In [44]:
model_fin.fit(
    train_ds,
    validation_data=test_ds,
    epochs=6,
    callbacks = [early_stop],
    shuffle = True,
)

Epoch 1/6
8/8 [==============================] - 5s 558ms/step - loss: 1.7328 - accuracy: 0.6494 - val_loss: 2.4738 - val_accuracy: 0.5455
Epoch 2/6
8/8 [==============================] - 4s 517ms/step - loss: 0.7820 - accuracy: 0.8367 - val_loss: 0.4544 - val_accuracy: 0.8485
Epoch 3/6
8/8 [==============================] - 4s 499ms/step - loss: 0.2502 - accuracy: 0.8924 - val_loss: 0.7266 - val_accuracy: 0.7879
Epoch 4/6
8/8 [==============================] - 4s 509ms/step - loss: 0.1472 - accuracy: 0.9482 - val_loss: 0.3224 - val_accuracy: 0.8939
Epoch 5/6
8/8 [==============================] - 4s 518ms/step - loss: 0.0766 - accuracy: 0.9761 - val_loss: 0.7249 - val_accuracy: 0.8030
Epoch 6/6
8/8 [==============================] - 4s 510ms/step - loss: 0.0658 - accuracy: 0.9641 - val_loss: 0.3037 - val_accuracy: 0.9242


In [45]:
y_pred = np.argmax(model_fin.predict(val_ds),axis=1)

3/3 [==============================] - 1s 219ms/step


In [46]:
train_label = np.concatenate([y for x, y in val_ds], axis=0)

test_label = np.concatenate([y for x, y in val_ds], axis=0) 

In [47]:
tf.math.confusion_matrix(labels=test_label,predictions=y_pred)

<tf.Tensor: shape=(3, 3), dtype=int32, numpy=
array([[26,  0,  0],
       [ 0, 16,  4],
       [ 0,  1, 19]])>